# Project task 01: Near duplicate detection with LSH

In [95]:

import gzip
import tarfile

import numpy as np
import pandas as pd
import time

from sklearn import preprocessing
from collections import defaultdict

import matplotlib.pyplot as plt
%matplotlib inline



SyntaxError: invalid syntax (<ipython-input-95-e0b47a77af94>, line 15)

To goal of this task is to find near duplicate songs in the Million Song dataset. You can imagine a scenario were the same song appears on multiple different releases with only small feature variation (e.g. duration or loudness).

## 1. Load data and extract the data

We'll be working with the Million Songs Dataset, a freely-available collection of audio features and metadata for a million contemporary popular music tracks.

Specifically, we will work with a smaller subset of 10 000 songs ([download link](http://static.echonest.com/millionsongsubset_full.tar.gz)).

In [67]:
tar = tarfile.open('millionsongsubset_full.tar.gz', 'r')
members = tar.getmembers()

In [68]:
tar.extract(members[5])
summary = pd.HDFStore(members[5].name)
songs = summary['/analysis/songs']

Show a snippet of how the data looks like:

In [69]:
songs.head()

,analysis_sample_rate,audio_md5,danceability,duration,end_of_fade_in,energy,idx_bars_confidence,idx_bars_start,idx_beats_confidence,idx_beats_start,...,key,key_confidence,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id
0,22050,a600d65cf157a306be60f26ecbf218f4,0.0,280.21506,0.238,0.0,0,0,0,0,...,5,0.555,-3.306,1,0.500,275.528,173.205,5,0.120,TRACCVZ128F4291A8A
1,22050,c64d75b0588e5ab552ee94548b50a4fa,0.0,191.68608,0.000,0.0,0,0,0,0,...,0,0.501,-10.764,1,0.710,184.128,150.955,4,0.600,TRACCMH128F428E4CD
2,22050,0cadd310865701bb93ced1cd78e8910a,0.0,222.92853,0.000,0.0,0,0,0,0,...,1,0.329,-9.035,1,0.407,216.300,93.056,4,1.000,TRACCSW128F148C7C3
3,22050,14be4fc1170152c445b3be7b8d18dfec,0.0,278.38649,0.496,0.0,0,0,0,0,...,7,0.313,-23.095,1,0.387,278.386,127.113,1,0.446,TRACCXJ128F428F0CF
4,22050,1def5d8298e8cb29a188a5a7c0e9429a,0.0,89.15546,4.847,0.0,0,0,0,0,...,2,0.000,-20.359,1,0.000,79.203,90.660,3,0.524,TRACCVS12903D014F8


We should have $31$ columns and $10~000$ rows.

In [70]:
print(len(songs))

10000


Since not all features are important we are going to consider a subset of features (columns) that are relevant for duplicate detection.

We will also convert the pandas dataframe into a numpy array so it is easier to work with.

In [71]:
subset = songs[['duration', 'end_of_fade_in', 'key', 'loudness',
                'mode', 'start_of_fade_out', 'tempo', 'time_signature',]]

data_matrix = subset.values

Additionally we will standardize the data to have zero mean and unit variance as a preprocessing step.

In [72]:
scaled_data = preprocessing.scale(data_matrix)

## 2. Implementaion

Your task is to implement near duplicate detection using LSH with cosine similarity.
More specifically you have to:
* Generate duplicate **candidates** based on LSH with $b$ bands and $r$ rows per band
* Refine the candidates by computing the exact cosine distance
* Report all pairs/duplicates with cosine distance < $d$

Implement a function that computes the cosine distance between two rows (instances) in the data.

In [73]:
def cosine_distance(X, i, j):
    """Compute cosine distance between two rows of a data matrix.
    
    Parameters
    ----------
    X : np.array, shape [N, D]
        Data matrix.
    i : int
        Index of the first row.
    j : int
        Index of the second row.
        
    Returns
    -------
    d : float
        Cosine distance between the two rows of the data matrix.
        
    """
    d = None
    
    c1 = X[i, :]
    c2 = X[j, :]
    
    d = 1 - np.dot(c1, c2) / (np.sqrt(np.dot(c1,c1)) * np.sqrt(np.dot(c2,c2)))
    
    return d

Cosine distance between the 5-th and the 28-th instance

In [74]:
print('{:.4f}'.format(cosine_distance(scaled_data, 5, 28)))

0.7273


In [98]:
def LSH(X, b=8, r=32, d=0.3):
    """Find candidate duplicate pairs using LSH and refine using exact cosine distance.
    
    Parameters
    ----------
    X : np.array shape [N, D]
        Data matrix.
    b : int
        Number of bands.
    r : int
        Number of rows per band.
    d : float
        Distance treshold for reporting duplicates.
    
    Returns
    -------
    duplicates : {(ID1, ID2, d_{12}), ..., (IDX, IDY, d_{xy})}
        A set of tuples indicating the detected duplicates.
        Each tuple should have 3 elements:
            * ID of the first song
            * ID of the second song
            * The cosine distance between them
    
    n_candidates : int
        Number of detected candidate pairs.
        
    """
    np.random.seed(158)
    n_candidates = 0
    duplicates = set()

    # number of songs
    n = len(X[:, 1])
    # number of attributes
    att = len(X[1, :])
    # number of hash functions
    m = b*r
    # initialize signature matrix
    M = np.array(np.zeros((m, n)))
    
    
    # create signature matrix M
    # for each hash function
    for i in range(m):
        # create random vector of 1s and 0s for each random projection
        v = np.random.randint(2, size=att)
        # compute hash value for every song
        for j in range(n):
            song = X[j, :]            
            dot = np.dot(v, song)
            M[i, j] = np.sign(dot)
#     print(M)
    
    
    # find candidates with signature matrix
    #hashing within buckets
    duplicate_index = set()
    # for every band
    for i in range(0, b*r, r):
        # create new buckets
        dictionary = defaultdict(list)
        # hash songs into a dict with hash value as key and position in X as value
        for j in range(n):
            band_signature = tuple(M[i:i+r, j])
#             print(M[i:i+r, j])
            # hash signature within band
            hash_key = hash(band_signature)
#             print(hash_key)
            dictionary[hash_key].append(j)
            
                   
        print("band")
        print(dictionary)
        # resulting dictionary = song's matrix index hashed into buckets within band

# haven't figured out the rest yet, but there is probably an error before since it doesn;t find candidates
        
        # what if more than 2 songs similar? hasn't happend yet
        if len(dictionary[hash_key]) > 2:
            print("more than 2 songs similar, what to do?")
             
        # check if there is more than one song in one bucket within a band 
        if len(dictionary[hash_key]) > 1:
            print("possible candidate found")
            indices = dictionary[hash_key]
            # check exact cosine distance 
            dist = cosine_distance(X, indices[0], indices[1])
            if dist <= d:
                # add indices to candidate indices
                duplicate_index.add(tuple(dictionary[hash_key]))
            
    
    print(duplicate_index)
                
        
    
    
    return duplicates, n_candidates

In [99]:
duplicates, n_candidates = LSH(scaled_data, b=3, r=64, d=0.0003)

band
defaultdict(<class 'list'>, {-5946744459574553069: [0, 70, 117, 174, 204, 263, 326, 336, 348, 353, 358, 369, 399, 415, 475, 513, 544, 601, 608, 613, 621, 657, 736, 752, 755, 855, 906, 952, 962, 1101, 1152, 1167, 1213, 1230, 1239, 1266, 1291, 1321, 1355, 1373, 1379, 1404, 1420, 1435, 1436, 1437, 1534, 1568, 1588, 1594, 1606, 1626, 1667, 1710, 1713, 1806, 1818, 1823, 1857, 1878, 1885, 1899, 1906, 1912, 1946, 1997, 2026, 2028, 2090, 2098, 2125, 2130, 2158, 2251, 2294, 2301, 2325, 2438, 2460, 2498, 2512, 2530, 2616, 2644, 2667, 2678, 2684, 2701, 2725, 2750, 2754, 2764, 2768, 2777, 2901, 2915, 2919, 2944, 2974, 2980, 3001, 3025, 3070, 3086, 3102, 3174, 3189, 3201, 3228, 3248, 3271, 3278, 3289, 3295, 3368, 3379, 3394, 3402, 3425, 3454, 3480, 3482, 3534, 3548, 3550, 3559, 3588, 3649, 3663, 3681, 3705, 3708, 3743, 3767, 3916, 3925, 3937, 3944, 3994, 4005, 4006, 4008, 4031, 4040, 4044, 4065, 4070, 4076, 4091, 4106, 4127, 4148, 4177, 4184, 4194, 4195, 4202, 4243, 4261, 4265, 4270, 4285, 428

In [ ]:
print('We detected {} candidates.'.format(n_candidates))

Show the duplicates we have found:

In [ ]:
duplicates

Show the metadata for the songs that were detected as duplicates:

In [ ]:
for i, j, d in duplicates:
    print('Song ID 1: {}'.format(i),
          'Song ID 2: {}'.format(j),
          'Distance: {:.6f}'.format(d),
          summary['/metadata/songs'].loc[i][['title', 'artist_name']].str.cat(sep=' - '),
          summary['/metadata/songs'].loc[j][['title', 'artist_name']].str.cat(sep=' - '), sep='\n')
    print()

## 3. Compare runtime

Your task is to implement code for runtime comparison between LSH and the naive nested for loop implementation.

In [ ]:
# naively compute the duplicates using a double for loop
def naive_duplicates(X, d = 0.2):
    """
    Parameters
    ----------
    X : np.array, shape [N, D]
        Data matrix.
    d : float
        Distance treshold for reporting duplicates.
    
    Returns
    -------
    duplicates : {(ID1, ID2, d_{12}), ..., (IDX, IDY, d_{xy})}
        A set of tuples indicating the detected duplicates.
        Each tuple should have 3 elements:
            * ID of the first song
            * ID of the second song
            * The cosine distance between them
    """
    N = X.shape[0]
    duplicates = set()
    for i in range(N):
        for j in range(N):
            d_ij = cosine_distance(X, i, j)
            if d_ij < d and i != j:
                duplicates.add((i, j, d_ij))
    return duplicates

In [ ]:
def runtime_comparison():
    """
    Compare the runtime between LSH and the naive approach.
    
    Returns
    -------
    trace : [(n1, lsh_dur, naive_dur), (n2, lsh_dur, naive_dur), ... ]
            A list of tuples with execution times for different number of songs.
            Each tuple should have 3 elements:
                * number of songs considered
                * duration of the LSH approach
                * duration of the naive approach
    """
    trace = []
    for n in np.arange(25, 251, 25):
        print('Running comparison for {} songs.'.format(n))
        
        ### YOUR CODE HERE ###
        
    return trace

In [ ]:
trace = runtime_comparison()

Plot the differecene in runtime. On the x-axis plot the number of songs processed and on the y-axis plot the runtime in seconds for both approaches. You should obtain a plot similar to the one shown below.

In [ ]:
### YOUR PLOTTING CODE HERE ###